In [1]:
import numpy as np
import scipy.sparse
from scipy.sparse.linalg import lobpcg, eigsh, minres, LinearOperator
from scipy.sparse import csr_matrix
import time

In [2]:
adj_path = "/Users/i.lobov/hyperwords/data/wiki/wikipedia.corpus.nodups_counts_win=1.adj"
adjacency_matrix = scipy.sparse.load_npz(adj_path + ".npz")
#adjacency_matrix.data = np.ones_like(adjacency_matrix.data, dtype=np.float64)
adjacency_matrix.data = adjacency_matrix.data ** 0.3
adjacency_matrix.data /= np.max(adjacency_matrix.data)

In [3]:
# n = adjacency_matrix.shape[0]
# degrees = np.asarray(adjacency_matrix.sum(axis=1)).flatten()
# D = scipy.sparse.spdiags(degrees, [0], n, n, format='csr')
# L = D - adjacency_matrix
# degrees_sqrt = np.sqrt(degrees)
# D_inv_sqrt = scipy.sparse.spdiags(1.0 / degrees_sqrt, [0], n, n, format='csr')
# L = D_inv_sqrt.dot(L.dot(D_inv_sqrt))

In [3]:
degrees = np.asarray(adjacency_matrix.sum(axis=1)).flatten()

In [4]:
# # PMI-adjacency
# n = adjacency_matrix.shape[0]
# degrees = np.asarray(adjacency_matrix.sum(axis=1)).flatten()
# total_count = degrees.sum()
# D_inv = scipy.sparse.spdiags(1.0 / degrees, [0], n, n, format='csr')
# adjacency_matrix = D_inv.dot(adjacency_matrix.dot(D_inv))
# adjacency_matrix.data = np.maximum(np.log(adjacency_matrix.data * total_count), 0)
# adjacency_matrix.eliminate_zeros()
# degrees = np.asarray(adjacency_matrix.sum(axis=1), dtype=np.float64).flatten() #Update degrees

In [8]:
#r = np.sqrt(5176.292450)
# r = np.sqrt(np.mean(degrees**2) / np.mean(degrees) - 1)
# n = adjacency_matrix.shape[0]

# dt = adjacency_matrix.data * r
# dt /= r**2 - adjacency_matrix.data ** 2

# adjacency_matrix.data = adjacency_matrix.data**2 / (r**2 - adjacency_matrix.data**2)
# bethe_diagonal = 1 + np.asarray(adjacency_matrix.sum(axis=1)).flatten()
# D = scipy.sparse.spdiags(bethe_diagonal, [0], n, n, format='csr')

# adjacency_matrix.data = dt
# I = scipy.sparse.eye(n, format='csr')
Hr = D - adjacency_matrix #+ I*np.mean(degrees)

In [13]:
preconditioner = scipy.sparse.spdiags(1.0 / bethe_diagonal, [0], n, n, format='csr')

In [9]:
dim = 100
tol = np.sqrt(1e-15)*n

start = time.time()
vals, vecs = eigsh(Hr, dim, which='SA', tol=tol)
print("time elapsed: %d" % (time.time() - start))

time elapsed: 162


In [10]:
vals #- np.mean(degrees)

array([-4.08467178, -0.15284486,  0.15484657,  0.22238634,  0.37784003,
        0.41869746,  0.46400135,  0.4742334 ,  0.5340904 ,  0.54998808,
        0.57052073,  0.59921006,  0.61810718,  0.63874438,  0.65217335,
        0.67033361,  0.69152106,  0.69664415,  0.70288655,  0.7119944 ,
        0.72597372,  0.73116844,  0.73374039,  0.74390417,  0.74672462,
        0.75024158,  0.75871329,  0.76157597,  0.76856867,  0.77332992,
        0.77360237,  0.77752917,  0.78226709,  0.78573242,  0.78627702,
        0.7908788 ,  0.79252999,  0.7958322 ,  0.79685831,  0.79995922,
        0.80563076,  0.80674431,  0.80930847,  0.80994591,  0.8131322 ,
        0.81528937,  0.81808489,  0.81988257,  0.82192655,  0.82221873,
        0.82545055,  0.82669662,  0.8289895 ,  0.82972457,  0.83124301,
        0.83375195,  0.83813602,  0.83861923,  0.83922756,  0.84080424,
        0.8425406 ,  0.84327683,  0.84479131,  0.84624515,  0.84754953,
        0.84879571,  0.84953354,  0.85068017,  0.8517487 ,  0.85

In [17]:
np.sqrt(1/np.abs(vals))

array([ 0.49479057,  2.55784714,  2.54126062,  2.12053747,  1.62684441,
        1.54543177,  1.46804841,  1.45212476,  1.36833556,  1.34841434,
        1.32392775,  1.29184513,  1.27194434,  1.251228  ,  1.23827891,
        1.2213904 ,  1.20253381,  1.19810396,  1.19277186,  1.18511832,
        1.17365256,  1.16947592,  1.16742446,  1.15942191,  1.15723021,
        1.15451462,  1.14805093,  1.1458912 ,  1.14066643,  1.13714958,
        1.13694931,  1.13407468,  1.13063511,  1.12813913,  1.12774837,
        1.12446264,  1.12329066,  1.12095776,  1.1202358 ,  1.11806249,
        1.11412002,  1.11335085,  1.11158571,  1.11114821,  1.10896904,
        1.10750095,  1.10560709,  1.10439434,  1.10302027,  1.10282427,
        1.10066326,  1.09983344,  1.09831138,  1.09782477,  1.09682161,
        1.09517007,  1.09230204,  1.09198731,  1.09159146,  1.0905675 ,
        1.08944317,  1.08896749,  1.08799095,  1.08705597,  1.08621915,
        1.08542148,  1.08495003,  1.08421858,  1.08353829,  1.08

array([ 3.16227766,  3.02891266,  2.91111255,  2.80606767,  2.71163072,
        2.62612866,  2.54823596,  2.47688702,  2.41121411,  2.35050247,
        2.29415734,  2.2416792 ,  2.19264505,  2.14669395,  2.10351581,
        2.06284249,  2.02444083,  1.98810693,  1.95366166,  1.92094688,
        1.88982237,  1.86016333,  1.83185826,  1.80480719,  1.77892017,
        1.75411604,  1.73032135,  1.70746944,  1.68549966,  1.66435666,
        1.64398987,  1.62435293,  1.60540325,  1.58710166,  1.56941205,
        1.55230105,  1.53573779,  1.51969366,  1.50414209,  1.48905839,
        1.47441956,  1.46020415,  1.44639214,  1.43296479,  1.41990459,
        1.40719509,  1.39482088,  1.38276747,  1.37102124,  1.35956936,
        1.34839972,  1.33750094,  1.32686223,  1.31647342,  1.30632486,
        1.29640745,  1.28671253,  1.27723192,  1.26795782,  1.25888286,
        1.25      ,  1.24130256,  1.23278418,  1.2244388 ,  1.21626064,
        1.20824419,  1.20038418,  1.19267561,  1.18511366,  1.17

In [14]:
# rng = np.random.RandomState(0)
# dim = 100
# init = rng.rand(n, dim)

# start = time.time()
# vals2, vecs2 = lobpcg(Hr, X=init, M=preconditioner, maxiter=100, largest=False, verbosityLevel=1)
# print("time elapsed: %d" % (time.time() - start))

Solving generalized eigenvalue problem with preconditioning

matrix size 189533
block size 100

No constraints


iteration 0
current block size: 100
eigenvalue: [ 0.55116048  0.99923643  0.99941171  0.99950066  0.99957271  0.99962842
  0.99964994  0.99966008  0.99967594  0.99971153  0.9997267   0.9997463
  0.99975736  0.99976402  0.99978327  0.99979215  0.99980889  0.99982053
  0.99983345  0.99985729  0.99988556  0.99989015  0.99989711  0.9999111
  0.9999248   0.99994209  0.99995184  0.99996482  0.99997484  0.99998932
  0.99999597  1.00001307  1.00002096  1.00002392  1.00003399  1.00004124
  1.00005054  1.00007132  1.00008407  1.0000882   1.00010463  1.00012276
  1.00012718  1.00014033  1.00014568  1.00015414  1.00016373  1.00016986
  1.00018174  1.00019153  1.00020624  1.00021085  1.00021785  1.0002305
  1.00023547  1.00024696  1.0002519   1.00027437  1.00028048  1.00028709
  1.00030363  1.00030428  1.00031902  1.00033709  1.00034523  1.00035219
  1.00036349  1.00037391  1.00038977  1

iteration 4
current block size: 89
eigenvalue: [-4.08466935 -0.15284185  0.15485301  0.2223884   0.37784452  0.41871397
  0.46401501  0.47425149  0.53412723  0.55004062  0.57058824  0.59932972
  0.61833223  0.63901611  0.65260162  0.67086072  0.69214184  0.69758982
  0.7036038   0.71283947  0.72745542  0.73280145  0.73487264  0.7453936
  0.74829899  0.75294026  0.76019123  0.7643448   0.7704916   0.77657429
  0.77775273  0.78053338  0.78600499  0.78860157  0.7917949   0.79730647
  0.79846938  0.80140876  0.80442662  0.80762523  0.81166366  0.81379448
  0.81812799  0.818702    0.82358679  0.82478422  0.82872083  0.83118968
  0.83429364  0.83510991  0.83701251  0.83913297  0.8408885   0.84414604
  0.84615692  0.84864993  0.85163656  0.85314029  0.8553456   0.85780269
  0.86121057  0.86416932  0.86533892  0.86648493  0.86942153  0.87008845
  0.87381887  0.87530197  0.87572547  0.87916295  0.88133035  0.88248677
  0.88316824  0.88672117  0.88841068  0.88945481  0.89144503  0.89276616
  0.8

iteration 8
current block size: 42
eigenvalue: [-4.08467135 -0.15284312  0.15485013  0.22238651  0.37784055  0.41869851
  0.46400248  0.47423519  0.53409314  0.54999142  0.57052758  0.59921068
  0.61810865  0.63874611  0.65217596  0.67034078  0.69152749  0.69665739
  0.70289552  0.71200599  0.72598511  0.73117192  0.73376076  0.7439126
  0.74672867  0.75028335  0.75871734  0.76158482  0.7685777   0.77334352
  0.77361652  0.77754091  0.78228376  0.78575745  0.7863019   0.79090826
  0.79254882  0.79584892  0.79687901  0.79999362  0.8056516   0.80678398
  0.80941939  0.80999492  0.81319634  0.81538267  0.81827778  0.81993891
  0.82198948  0.82225994  0.82554701  0.82676999  0.82909827  0.82979024
  0.83145965  0.83385093  0.83826423  0.83877183  0.83942002  0.84113386
  0.84286629  0.84343029  0.84516138  0.84654093  0.8477998   0.8495349
  0.85016695  0.85129478  0.85219851  0.8534619   0.85544106  0.856065
  0.85813282  0.85989176  0.86014034  0.8611971   0.86262026  0.86351373
  0.8638

iteration 12
current block size: 7
eigenvalue: [-4.08467136 -0.15284325  0.1548498   0.22238648  0.3778405   0.41869833
  0.46400239  0.47423499  0.5340929   0.54999112  0.57052651  0.5992106
  0.61810844  0.63874583  0.65217555  0.67033876  0.69152635  0.69665416
  0.70289364  0.71200402  0.72597896  0.7311712   0.73375736  0.74390933
  0.74672766  0.75025592  0.75871675  0.76158248  0.76857589  0.77333518
  0.77361305  0.77753725  0.78227929  0.78574588  0.78628777  0.7908891
  0.79253977  0.79584241  0.79686722  0.79997513  0.80564132  0.80676737
  0.80932955  0.80996655  0.81316428  0.81532332  0.8181191   0.81991441
  0.82194224  0.82223309  0.82547247  0.82673554  0.82902009  0.82974451
  0.83127499  0.83378381  0.83817532  0.83866033  0.83928281  0.84085986
  0.84258376  0.84331624  0.84483251  0.84629429  0.84763458  0.84889003
  0.849577    0.85078048  0.85181629  0.85278154  0.85501807  0.8555576
  0.85746457  0.8592143   0.85950571  0.85999301  0.86157777  0.86253299
  0.863

iteration 16
current block size: 2
eigenvalue: [-4.08467136 -0.15284326  0.15484977  0.22238648  0.37784049  0.41869831
  0.46400238  0.47423498  0.53409288  0.54999111  0.57052644  0.59921059
  0.61810843  0.63874582  0.65217553  0.67033866  0.6915263   0.69665403
  0.70289361  0.71200397  0.72597869  0.73117117  0.73375729  0.74390914
  0.74672758  0.75025516  0.75871673  0.76158235  0.76857582  0.77333494
  0.77361293  0.77753722  0.78227918  0.78574568  0.78628757  0.79088883
  0.79253941  0.79584226  0.79686699  0.79997449  0.80564102  0.80676679
  0.80932782  0.80996611  0.81316344  0.81532172  0.81811665  0.819914
  0.82194194  0.82223282  0.82547076  0.82673514  0.82901843  0.82974426
  0.83127204  0.83378293  0.83817443  0.83866     0.83928178  0.84085525
  0.84258098  0.84331547  0.84483168  0.84629196  0.84763244  0.84887257
  0.84956894  0.85076965  0.85181404  0.85276937  0.85501043  0.85555349
  0.85745632  0.8591937   0.85949049  0.85997877  0.86155774  0.86251935
  0.86

In [15]:
vals2 #- np.mean(degrees)

array([-4.08467136, -0.15284326,  0.15484977,  0.22238648,  0.37784049,
        0.41869831,  0.46400238,  0.47423498,  0.53409288,  0.54999111,
        0.57052644,  0.59921059,  0.61810843,  0.63874582,  0.65217553,
        0.67033866,  0.6915263 ,  0.69665403,  0.7028936 ,  0.71200396,
        0.72597869,  0.73117117,  0.73375729,  0.74390914,  0.74672758,
        0.75025516,  0.75871673,  0.76158234,  0.76857582,  0.77333493,
        0.77361292,  0.77753722,  0.78227917,  0.78574567,  0.78628757,
        0.79088882,  0.79253939,  0.79584226,  0.79686698,  0.79997449,
        0.80564101,  0.80676678,  0.80932782,  0.80996607,  0.81316342,
        0.81532171,  0.8181166 ,  0.81991398,  0.82194189,  0.82223281,
        0.82547076,  0.82673502,  0.82901823,  0.82974425,  0.83127203,
        0.83378292,  0.83817431,  0.83865991,  0.83928177,  0.84085514,
        0.84258098,  0.84331547,  0.84483167,  0.84629183,  0.84763242,
        0.84887243,  0.84956875,  0.85076912,  0.85181384,  0.85

In [ ]:
# SA - 249 secs
# LM + sigma (-51) + iterative inverse - 528

In [13]:
vals

array([-4.08467178, -0.15284486,  0.15484657,  0.22238634,  0.37784003,
        0.41869746,  0.46400135,  0.4742334 ,  0.5340904 ,  0.54998808,
        0.57052073,  0.59921006,  0.61810718,  0.63874438,  0.65217335,
        0.67033361,  0.69152106,  0.69664415,  0.70288655,  0.7119944 ,
        0.72597372,  0.73116844,  0.73374039,  0.74390417,  0.74672462,
        0.75024158,  0.75871329,  0.76157597,  0.76856867,  0.77332992,
        0.77360237,  0.77752917,  0.78226709,  0.78573242,  0.78627702,
        0.7908788 ,  0.79252999,  0.7958322 ,  0.79685831,  0.79995922,
        0.80563076,  0.80674431,  0.80930847,  0.80994591,  0.8131322 ,
        0.81528937,  0.81808489,  0.81988257,  0.82192655,  0.82221873,
        0.82545055,  0.82669662,  0.8289895 ,  0.82972457,  0.83124301,
        0.83375195,  0.83813602,  0.83861923,  0.83922756,  0.84080424,
        0.8425406 ,  0.84327683,  0.84479131,  0.84624515,  0.84754953,
        0.84879571,  0.84953354,  0.85068017,  0.8517487 ,  0.85

In [13]:
output_path = "../data/wiki/win=1_weighted_bethe_hessian_scaled2_pow=0.30_dim=100"
np.save(output_path + ".vecs", vecs)
np.save(output_path + ".vals", vals - np.mean(degrees))
np.save(output_path + ".degrees", degrees)

In [14]:
output_path = "../data/wiki/win=1_weighted_bethe_hessian_scaled3_pow=0.30_dim=100"
np.save(output_path + ".vecs", vecs2)
np.save(output_path + ".vals", vals2 - np.mean(degrees))
np.save(output_path + ".degrees", degrees)

In [11]:
output_path = "../data/wiki/win=1_bethe_hessian_small_rhoB_est_pow=0.00_dim=100"
vecs2 = np.load(output_path + ".vecs.npy")
vals2 = np.load(output_path + ".vals.npy")

In [14]:
all_vals = np.concatenate([vals, vals2], axis=0)
all_vecs = np.concatenate([vecs, vecs2], axis=1)

In [19]:
top_vals_inds = np.argsort(all_vals)[:100]
output_path = "../data/wiki/win=1_bethe_hessian_combo_rhoB_est_pow=0.00_dim=100"
np.save(output_path + ".vecs", all_vecs[:, top_vals_inds])
np.save(output_path + ".vals", all_vals[top_vals_inds])
np.save(output_path + ".degrees", degrees)